In [1]:
pip install lxml aiohttp asyncio nest_asyncio aiofiles

     |████████████████████████████████| 4.6 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 574 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 5.3 MB/s ta 0:00:011
     |████████████████████████████████| 121 kB 4.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import shutil
from lxml import etree
from lxml import html
import aiohttp
import asyncio
import aiofiles
import nest_asyncio
import os

#fix for running in colab
nest_asyncio.apply()

#all categories will be compressed into storeDirectory + download.zip on completion

#ONLY CHNAGE THIS
url = 'https://commons.wikimedia.org/wiki/Category:ICE'
storeDirectory = '/content/'
checkForCategories = True

#DON'T CHANGE
tasks = []
categories = 0
categoryTasks = []
checkedCategories = []
completed = -1
totalImages = 0
completedImages = 0

async def fetch_page(session, url, cat = ''):
  try:
    async with session.get(url) as resp:
      source = await resp.text()

      dom = html.fromstring(source)

      return [cat, dom]
  except asyncio.TimeoutError or aiohttp.ClientConnectorError:
    #print('Timeout')
    return False

async def fetch_images(session, url):
  global totalImages

  dom = await fetch_page(session, url)

  #timeout error
  if dom == False:
    return

  images = dom[1].xpath('*//div[@class="thumb"]//a')
  subcategories = dom[1].xpath('*//div[@class="CategoryTreeItem"]//a')

  if(len(subcategories) > 0 and checkForCategories):
    for category in subcategories:
      if(category not in checkedCategories):
        categoryTasks.append(asyncio.ensure_future(fetch_images(session, 'https://commons.wikimedia.org' + category.attrib['href'])))
        checkedCategories.append(category)
        print('Found category', category.attrib['href'])

  if (len(images) > 0):
    totalImages += len(images)
    print("Found", len(images), "images")
    #download images for each category
    for image in images:
      cat = url.split('Category:')[1]
      tasks.append(asyncio.ensure_future(fetch_page(session, 'https://commons.wikimedia.org' + image.attrib['href'], cat)))

  global completed
  completed += 1

async def main(loop):
  global url
  global completedImages

  async with aiohttp.ClientSession(loop=loop) as session:
    await fetch_images(session, url)

    #fix to resolve finding all categories first
    while True:
      await asyncio.gather(*categoryTasks)

      #check if images have been found on all category pages
      if(completed == len(categoryTasks)):
        break

    pages = await asyncio.gather(*tasks)
    for page in pages:
      #timeout error
      if(page == False):
        continue

      cat = page[0]
      source = page[1]
       
      #print(cat, source.xpath('*//div[@class="fullImageLink"]//img')[0].attrib['src'])
      imgURL = source.xpath('*//div[@class="fullImageLink"]//img')[0].attrib['src']

      filename = imgURL.split('/')[-1]
      #TODO: save images into category folders
      async with session.get(imgURL) as resp:
        if resp.status == 200:
            if(os.path.isdir(storeDirectory + cat + '/') == False):
              os.mkdir(storeDirectory + cat + '/')

            f = await aiofiles.open(storeDirectory + cat + '/' + filename, mode='wb')
            await f.write(await resp.read())
            await f.close()
            completedImages += 1
            print(completedImages, '/', totalImages)

  #create zip file to download
  shutil.make_archive(storeDirectory + 'download', 'zip', storeDirectory)

#main event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

Found category /wiki/Category:ICE_trains_in_Belgium
Found category /wiki/Category:ICE_trains_in_France
Found category /wiki/Category:ICE_trains_in_Germany
Found category /wiki/Category:ICE_trains_in_the_Netherlands
Found category /wiki/Category:DBAG_Class_401
Found category /wiki/Category:DBAG_Class_402
Found category /wiki/Category:DBAG_Class_403/406
Found category /wiki/Category:NS_Class_406
Found category /wiki/Category:DBAG_Class_407
Found category /wiki/Category:DBAG_Class_408
Found category /wiki/Category:DBAG_Class_412
Found category /wiki/Category:ICE_100
Found category /wiki/Category:ICE_1000
Found category /wiki/Category:ICE_1100
Found category /wiki/Category:ICE_1579
Found category /wiki/Category:ICE_1609
Found category /wiki/Category:ICE_222
Found category /wiki/Category:ICE_374
Found category /wiki/Category:ICE_506
Found category /wiki/Category:ICE_583
Found category /wiki/Category:ICE_606
Found category /wiki/Category:ICE_620
Found category /wiki/Category:ICE_712
Found ca

KeyboardInterrupt: 

Found category /wiki/Category:2008_Landr%C3%BCckentunnel_rail_accident
Found category /wiki/Category:ICE_885
Found 4 images
Found category /wiki/Category:ICE_885
Found 4 images
Found category /wiki/Category:2008_Landr%C3%BCckentunnel_rail_accident
Found category /wiki/Category:ICE_885
Found 4 images
Found category /wiki/Category:2008_Landr%C3%BCckentunnel_rail_accident
Found category /wiki/Category:2008_Landr%C3%BCckentunnel_rail_accident
Found category /wiki/Category:ICE_885
Found 4 images
Found category /wiki/Category:ICE_885
Found 4 images
Found category /wiki/Category:2008_Landr%C3%BCckentunnel_rail_accident
Found category /wiki/Category:2008_Landr%C3%BCckentunnel_rail_accident
Found category /wiki/Category:ICE_885
Found 4 images
Found category /wiki/Category:ICE_885
Found 4 images
Found category /wiki/Category:2008_Landr%C3%BCckentunnel_rail_accident
Found category /wiki/Category:2008_Landr%C3%BCckentunnel_rail_accident
Found category /wiki/Category:ICE_885
Found 4 images
Found ca